In [1]:
import string
import pandas as pd
from nltk.corpus import stopwords

In [2]:
# set up emolex analysis
emolex_df = pd.read_csv('data/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', sep='\t', names=['word', 'emotion','association'])
emolex_df = emolex_df[emolex_df.association == 1]
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association')
emolex_words = emolex_words.reset_index()

In [3]:
def count_emotions(text):

    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    words = text.split()
    stop_words = set(stopwords.words('english'))

    words = [word for word in words if word not in stop_words]
    emotions_count = emolex_words[emolex_words.word.isin(words)].sum()
    emotions_count['word_count'] = len(words)

    return emotions_count

In [4]:
# read the restaurant dataset with vader scores
df = pd.read_csv('data/vader.csv')

In [5]:
# run the emolex analysis on the dataframe and add to the dataframe
emotions_count = df['text'].apply(count_emotions)
vader_emolex_rest = pd.concat([df, emotions_count], axis=1)

In [6]:
# set index to review id and drop columns not required
vader_emolex_rest = vader_emolex_rest.reset_index()
vader_emolex_rest = vader_emolex_rest.set_index('review_id')
vader_emolex_rest.drop(['index','word'], axis=1, inplace=True)

In [7]:
# vader_emolex_rest.to_json('data/vader_emolex_rest.json')
vader_emolex_rest.to_csv('data/vader_emolex.csv')